# CO_2 and Weather

## Formalia:

What are we trying to figure out

## Part 1: Development of CO2 emission in the world

In [ ]:
EU = ['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden'] 

: 

## Generating plots for Part 1


In [ ]:
# Import libraries
from math import ceil
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import geojson
#Only two digit after the decimal point
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning

In [ ]:
df= pd.read_csv('annual-co2-emissions-per-country.csv')
#replace NaN with 0
df = df.fillna(0)  
#replace inf with 0
df = df.replace([np.inf, -np.inf], 0)

# if Slovak Republic then replace with Slovakia
df.replace(to_replace='Slovak Republic', value='Slovakia', inplace=True)
# Czech Republic to Czechia
df.replace(to_replace='Czech Republic', value = 'Czechia', inplace=True)
df = df.drop(columns=['Code'])
df = df.rename(columns={'Entity': 'Country Name'})

df.head()

In [ ]:
df = df.pivot(index='Country Name', columns='Year', values='Annual CO2 emissions')
df = df.reset_index()
df = df.rename(columns={'Country Name': 'Country Name'})
df = df.set_index('Country Name')
df = df.fillna(0)
# remove the word Year, so Country moved one up
df.columns.name = None
df = df.reset_index()
#

df.head()

In [ ]:
col_list = []

for year in range(1750, 1960):
    col_list.append(int(year))

# drop column thats in the col_list
df.drop(columns=col_list, axis=1, inplace=True)
df.head()

In [ ]:
df_EU = df.copy()
df_EU = df_EU[df_EU['Country Name'].isin(EU)]

df_EU.head(19)

In [ ]:
# Some basis statistics using stats for EU CO2 emission
df_EU.describe()


In [ ]:
# Some basis statistics using stats for EU CO2 emission
decribe_stat = df_EU.describe()
# plot the df_EU.describe() each stat in a subplot
fig, ax = plt.subplots(2, 3, figsize=(27, 12))
fig.suptitle('Statistics for EU CO2 Emission')
decribe_stat.loc['mean'].plot(kind='bar', ax=ax[0, 0], title='Mean',grid=True, yticks=range(0, 9))
decribe_stat.loc['std'].plot(kind='bar', ax=ax[0, 1], title='Standard Deviation',grid=True, yticks=range(0, 9))
decribe_stat.loc['min'].plot(kind='bar', ax=ax[0, 2], title='Min',grid=True, yticks=range(0, 5))
decribe_stat.loc['25%'].plot(kind='bar', ax=ax[1, 0], title='25%',grid=True, yticks=range(0, 7))
decribe_stat.loc['50%'].plot(kind='bar', ax=ax[1, 1], title='50%',grid=True, yticks=range(0, 9))
decribe_stat.loc['75%'].plot(kind='bar', ax=ax[1, 2], title='75%',grid=True, yticks=range(0, 12))

plt.show()
#save the figure
fig.savefig('EU_CO2_Emission_Statistics.png')


In [ ]:
df_EU_yearly = df.copy()
df_EU_yearly = df_EU.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
df_EU_yearly.head()


In [ ]:
# First first row is the year and first coloumn is the country name, make a plot where the x-axis is the year and the y-axis is the CO2 emission and make it so theres a subplot for each country
col = 3
rows = math.ceil(len(EU) / col)


fig, ax = plt.subplots(rows, col, sharex=True, figsize=(24, 15))

for index, country in enumerate(EU):
    plt.subplot(rows, col, index+1)
    plt.title(country)#, x = 0.1, y=0.7, pad=20) 
    plt.xlabel('Year')
    plt.ylabel('CO2 Emission')
    y_min, y_max = 0, df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'].max()   
    #print(f'{country} --> y_min: {y_min}, y_max: {y_max}')
    plt.ylim([y_min, y_max])
    y_interval = int((y_max - y_min) / 5) if y_max > 5 else 5
    plt.yticks(range(0, int(y_max), y_interval))
    plt.xticks(rotation=90)
    # Bar chart for each country with the CO2 emission for each year
    plt.bar(df_EU_yearly[df_EU_yearly['Country Name'] == country]['Year'], df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'])
   
    plt.tight_layout()
    plt.title('CO2 Emission for ' + country)
plt.tight_layout()

# save to html
plt.savefig('CO2_Emission_EU.png')
fig.suptitle('CO2 Emission for each country in the EU distribuated through the years 1960-2018', fontsize=16, y=1.05)
plt.show()



In [ ]:
# Total emission for world for each year
df_world = df.copy()
# should only have the Country World
df_world = df_world[df_world['Country Name'] == 'World']
# drop the Country World
df_world = df_world.sum(axis=0)
df_world = df_world.reset_index()
df_world.columns = ['Year', 'CO2 Emission']
#remove max_year and year_max
df_world = df_world.drop(df_world[df_world['Year'] == 'max_year'].index)
df_world = df_world.drop(df_world[df_world['Year'] == 'year_max'].index)
#plot the total CO2 emission for the world
fig = px.line(df_world, x='Year', y='CO2 Emission',title='CO2 Emission per year for the world from 1960-2022')
fig.show()
fig.write_html("CO2_Emission_world_1960_2022.html")

In [ ]:
# Top 10 list with highest CO2 emission in 2018
top_5 = ['China', 'United States', 'India', 'Russia', 'Japan']
highest_country = df.copy()
highest_country =  highest_country[highest_country['Country Name'].isin(top_5)]
highest_country =highest_country.reset_index()
highest_country = highest_country.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')


df_EU_total = df_EU.copy()
df_EU_total = df_EU_total.drop(columns=['Country Name'])
df_EU_total = df_EU_total.sum(axis=0)
df_EU_total = df_EU_total.reset_index()
df_EU_total.columns = ['Year', 'CO2 Emission']
df_EU_total['Country Name'] = 'EU'


df_combine = pd.concat([df_EU_total, highest_country])

print(df_combine)

#remove column with NaN
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'max_year'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_max'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_min'].index)

#plot the total CO2 emission for the world
fig = px.line(df_combine, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for the world from 1960-2018')
fig.show()
fig.write_html("CO2_Emission_world_1960_2018.html")

In [ ]:
# First first row is the year and first coloumn is the country name, make a plot where the x-axis is the year and the y-axis is the CO2 emission and make it so theres a subplot for each country
fig = px.line(df_EU_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the EU')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_EU.html")


In [ ]:
# Timeserie with world co2 emission
df_world = df.copy()
df_world = df_world[~df_world['Country Name'].isin(EU)]
df_world_yearly = df_world.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
#plot the world co2 emission
fig = px.line(df_world_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the world')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_World.html")

In [ ]:
df['max_year'] = df.select_dtypes(include=[np.number]).max(axis=1)
df['year_max'] = df.select_dtypes(include=[np.number]).idxmax(axis=1)
#df['min_year'] = df.min(axis=1)

for i in range(4, 60):
    df.loc[df['year_max'] == df.columns[i], 'year_max'] = df.columns[i]

# plotly bar plot witg country name, max_year and year_max
fig = go.Figure(data=[go.Table(
    header=dict(values=['Country Name', 'Max CO2 Emission', 'Year of Max CO2 Emission']),
    cells=dict(values=[df['Country Name'], df['max_year'], df['year_max']])
)])
fig.update_layout(title='Max CO2 Emission and Year of Max CO2 Emission for countries')
#fig.write_html("Max_CO2_Emission_and_Year_of_Max_CO2_Emission_for_countries.html")
fig.show()
#save as html
fig.write_html("Max_CO2_Emission_and_Year_of_Max_CO2_Emission_for_countries.html")


In [ ]:
df_EU['max_year'] = df_EU.select_dtypes(include=[np.number]).max(axis=1)
df_EU['min_year'] = df_EU.select_dtypes(include=[np.number]).min(axis=1)
df_EU['year_max'] = df_EU.select_dtypes(include=[np.number]).idxmax(axis=1)
df_EU['year_min'] = df_EU.select_dtypes(include=[np.number]).idxmin(axis=1)


# plotly bar plot with country name, max_year, min_year, year_max and year_min
fig = go.Figure(data=[go.Table(
    header=dict(values=['Country Name', 'Max CO2 Emission', 'Year of Max CO2 Emission', 'Min CO2 Emission', 'Year of Min CO2 Emission']),
    cells=dict(values=[df_EU['Country Name'], df_EU['max_year'], df_EU['year_max'], df_EU['min_year'], df_EU['year_min']])
)])
fig.update_layout(title='Max and Min CO2 Emission and Year of Max and Min CO2 Emission for EU countries')
fig.write_html("List_Max_and_Min_CO2_Emission_and_Year_of_Max_and_Min_CO2_Emission_for_EU_countries.html")
fig.show()
print(f'First part')

In [ ]:
# plotly bar chart with country name, max_year, year_max, min_year and year_min
fig = go.Figure(data=[ go.Bar(name='Min CO2 Emission', x=df_EU['Country Name'], y=df_EU['min_year'], text=df_EU['year_min'], textposition='auto'),
                      go.Bar(name='Max CO2 Emission', x=df_EU['Country Name'], y=df_EU['max_year'], text=df_EU['year_max'], textposition='auto')])
fig.update_layout(title='Max and Min CO2 Emission and Year of Max and Min CO2 Emission for EU countries', barmode='group')
fig.write_html("Bar_chart_Max_and_Min_CO2_Emission_and_Year_of_Max_and_Min_CO2_Emission_for_EU_countries.html")
fig.show()

In [ ]:
df_EU_yearly.head()

In [ ]:
# max value in de_EU_yearly
max_value = df_EU_yearly['CO2 Emission'].max()
# min value in df_EU_yearly
min_value = df_EU_yearly['CO2 Emission'].min()

In [ ]:
with  open('europe.geojson') as response: #### Using the geojson of lat and lon of San Fransisco
    counties = geojson.load(response) 

fig = px.choropleth(df_EU_yearly, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", ## ????
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           #locationmode='geojson-id',
                           range_color=(min_value, max_value),
                           #mapbox_style="carto-positron",
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           #opacity=1,
                           #labels={'CO2 Emission':'CO2 Emission'}
                           width=1000, height=800,
                           title = 'CO2 for EU from 1960-2018'
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#save as html
fig.write_html("CO2_Emission_EU_Map_Online.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""

## Part 2: 
## Exploring CO2 Emission in Europe: A look into the history of Highs and Lows
The following is focusing on visiualisations that shows the highs and lows of the co2 emission history in Europe.
Line plots, and visualisation of geodata using plotly.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, find_peaks, find_peaks, find_peaks

# Only two digits after the decimal point
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Filter data for the years between 1900 and 2022
df_EU_yearly = df_EU_yearly[(df_EU_yearly['Year'] >= 1900) & (df_EU_yearly['Year'] <= 2022)]

# Total CO2 emissions for each year
total_CO2_each_year = df_EU_yearly.groupby('Year')['CO2 Emission'].sum()

# Find peaks in the data
peaks, _ = find_peaks(total_CO2_each_year.values)
valleys, _ = find_peaks(-total_CO2_each_year.values)

# Plot the total CO2 emission for each year
plt.figure(figsize=(15, 6))
plt.plot(total_CO2_each_year.index, total_CO2_each_year.values, linestyle='-')

# Plot peaks and valleys
plt.plot(total_CO2_each_year.index[peaks], total_CO2_each_year.values[peaks], marker='o', linestyle='', color='red', markersize=8)
plt.plot(total_CO2_each_year.index[valleys], total_CO2_each_year.values[valleys], marker='o', linestyle='', color='green', markersize=8)

plt.xlabel('Year')
plt.ylabel('Total CO2 Emission')
plt.title('Total CO2 Emission for each year (EU Countries)')
plt.xticks(total_CO2_each_year.index, rotation=90)  # Set x-axis ticks every year
plt.xlim(1960, 2022)
plt.grid()
plt.savefig('Total_CO2_Emission_for_each_year_EU.png')
plt.show()


: 

This is a linear plot that showcases the total amount of CO2 emission in europe each year. We do see a high increase of CO2 emission in year 1979, while the lowest CO2 emission after 1979 was in year 2020. 

In [ ]:
# Filter data for the year 1990
# Filter data for the year 1990
df_1979 = df_EU_yearly[df_EU_yearly['Year'] == 1979]

# max value in de_EU_yearly
max_value = df_EU_yearly['CO2 Emission'].max()
# min value in df_EU_yearly
min_value = df_EU_yearly['CO2 Emission'].min()

with  open('europe.geojson') as response: #### Using the geojson of lat and lon of San Fransisco
    counties = geojson.load(response) 

fig = px.choropleth(df_1979, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", ## ????
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           #locationmode='geojson-id',
                           range_color=(min_value, max_value),
                           #mapbox_style="carto-positron",
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           #opacity=1,
                           #labels={'CO2 Emission':'CO2 Emission'}
                           width=700, height=500,
                           title = 'CO2 for EU from 1960-2018'
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#save as html
fig.write_html("CO2_Emission_EU_Map_Online.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""

The colors on the map showcases the CO2 emission in EU in 1990. The reasoning behind showcasing CO2 emission for that year is because it is the year that EU experienced the most CO2 emission as seen in the Linear plot. When zooming in, we do see that Luxemborg is the only red country on the map, which means that Luxemborg is most likely to be the country in 1990 with the highest CO2 emission. Furthermore we see that Portugal is the country that is the most dark blue country on the map, and therefor is the country that most likely had the lowest CO2 emission in 1990. To make sure that these observations are correct, we just cross checked by doing the following. 

In [ ]:
#Highest CO2 emission in 1990
highest_emission_row = df_1979.loc[df_1979['CO2 Emission'].idxmax()]

# Extract the country name and its corresponding CO2 emission value
highest_emission_country = highest_emission_row['Country Name']
highest_emission_value = highest_emission_row['CO2 Emission']

# Lowest CO2 emission in 1990
lowest_emission_row = df_1979.loc[df_1979['CO2 Emission'].idxmin()]

# Extract the country name and its corresponding CO2 emission value
lowest_emission_country = lowest_emission_row['Country Name']
lowest_emission_value = lowest_emission_row['CO2 Emission']

print("Country with the highest CO2 emission in 1990:", highest_emission_country)
print("CO2 emission value:", highest_emission_value)

print("\nCountry with the lowest CO2 emission in 1990:", lowest_emission_country)
print("CO2 emission value:", lowest_emission_value)


As seen in the print out statements, Luxemborg is the country that had the highest CO2 emission in year 1990, while Portugal was the country with the lowest CO2 emission. Another interesting perspective was to look into 2018 and see which country had the highest CO2 emission and the country with the CO2 emission with the lowest CO2 emission.

In [ ]:
df_2020 = df_EU_yearly[df_EU_yearly['Year'] == '2020']
# Highest CO2 emission in 1990
highest_emission_row = df_2020.loc[df_2020['CO2 Emission'].idxmax()]

# Extract the country name and its corresponding CO2 emission value
highest_emission_country = highest_emission_row['Country Name']
highest_emission_value = highest_emission_row['CO2 Emission']

# Lowest CO2 emission in 1990
lowest_emission_row = df_2020.loc[df_2020['CO2 Emission'].idxmin()]

# Extract the country name and its corresponding CO2 emission value
lowest_emission_country = lowest_emission_row['Country Name']
lowest_emission_value = lowest_emission_row['CO2 Emission']

print("Country with the highest CO2 emission in 2020:", highest_emission_country)
print("CO2 emission value:", highest_emission_value)

print("\nCountry with the lowest CO2 emission in 2020:", lowest_emission_country)
print("CO2 emission value:", lowest_emission_value)


: 

Luxemborg still remains as the EU country with the highest CO2 emission, while Malta in 2018 is now the EU country with the lowest CO2 emission. 

In [ ]:
df_2015 = df_EU_yearly[df_EU_yearly['Year'] == '2015']
# Highest CO2 emission in 1990
highest_emission_row = df_2015.loc[df_2015['CO2 Emission'].idxmax()]

# Extract the country name and its corresponding CO2 emission value
highest_emission_country = highest_emission_row['Country Name']
highest_emission_value = highest_emission_row['CO2 Emission']

# Lowest CO2 emission in 1990
lowest_emission_row = df_2015.loc[df_2015['CO2 Emission'].idxmin()]

# Extract the country name and its corresponding CO2 emission value
lowest_emission_country = lowest_emission_row['Country Name']
lowest_emission_value = lowest_emission_row['CO2 Emission']

print("Country with the highest CO2 emission in 2015:", highest_emission_country)
print("CO2 emission value:", highest_emission_value)

print("\nCountry with the lowest CO2 emission in 2015:", lowest_emission_country)
print("CO2 emission value:", lowest_emission_value)

As seen in the line plot earlier, the year that had the lowest CO2 emission after 1990 was year 2015. The above showcases the highest and lowest CO2 emission by a country.

Luxemborg had once again the highest CO2 emission, while Romania had the lowest CO2 emission that year

## Part 3: Extreme weather 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_file
from bokeh.models import Legend, HoverTool, ColumnDataSource
from bokeh.layouts import gridplot


# For EU dataset 
excel_file_EU = 'EU_disasters.xlsx'

df = pd.read_excel(excel_file_EU)

df.drop(columns=['Historic', 'Classification Key','External IDs', 'Event Name','OFDA/BHA Response','Appeal','Declaration',"AID Contribution ('000 US$)", 'Magnitude','Magnitude Scale', 'Latitude','Longitude','River Basin','Start Day','Total Deaths','No. Injured','No. Affected','No. Homeless','Total Affected',"Reconstruction Costs ('000 US$)", 
                 "Reconstruction Costs, Adjusted ('000 US$)","Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", "Total Damage ('000 US$)","Total Damage, Adjusted ('000 US$)",'CPI', "Admin Units","Entry Date", 'Last Update','Origin','Associated Types'], inplace=True)


df['Start Year'] = pd.to_datetime(df['Start Year'], format='%Y').dt.year

# For WorldWide dataset
excel_file_World = 'WW.xlsx'

df_world = pd.read_excel(excel_file_World)
df_world.drop(columns=['Historic', 'Classification Key','External IDs', 'Event Name','OFDA/BHA Response','Appeal','Declaration',"AID Contribution ('000 US$)", 'Magnitude','Magnitude Scale', 'Latitude','Longitude','River Basin','Start Day','Total Deaths','No. Injured','No. Affected','No. Homeless','Total Affected',"Reconstruction Costs ('000 US$)", 
                 "Reconstruction Costs, Adjusted ('000 US$)","Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", "Total Damage ('000 US$)","Total Damage, Adjusted ('000 US$)",'CPI', "Admin Units","Entry Date", 'Last Update','Origin','Associated Types'], inplace=True)


df_world['Start Year'] = pd.to_datetime(df['Start Year'], format='%Y').dt.year



# global values
generic_colors = ['red', 'yellow', 'purple', 'blue', 'green', 'black', 'gray', 'pink', 'brown',
                  'skyblue', 'darkorange', 'lightseagreen', 'cyan', 'magenta', 'lime', 'gold',
                  'indigo', 'teal', 'salmon', 'peru', 'olive', 'orchid', 'steelblue','black']

disaster_colors = {
    "Flood": "blue",
    "Storm": "red",
    "Earthquake": "green",
    "Epidemic": "orange",
    "Glacial lake outburst flood": "purple",
    "Extreme temperature" :"magenta",
    "Mass movement (wet)":"lime",
    "Volcanic activity":"indigo",
    "Wildfire":"steelblue",
    "Drought":"peru",
    "Impact": "teal",
    "Infestation":"olive",
    "Mass movement (dry)":"magenta",
}


In [ ]:
###### Filter the two dataset to only consider the natural disasters ######


# For EU dataset 
filter_natural = df[df['Disaster Group'] == 'Natural']

grouped_natural_disaster = filter_natural.groupby('Disaster Group')

# For EU dataset 

filter_natural_world = df_world[df_world['Disaster Group'] == 'Natural']

grouped_natural_disaster_world = filter_natural_world.groupby('Disaster Group')

In [ ]:
# Plot for the sum of all disasters each year 

# For EU
grouped_start_year = filter_natural.groupby('Start Year').size()

plt.figure(figsize=(20, 8))

plt.plot(grouped_start_year.index, grouped_start_year.values)

plt.xlabel('Start Year')
plt.ylabel('Number of Natural Disasters in EU')
plt.title('Number of Natural Disasters in EU in each year')

plt.savefig("natural_disasters_all_year_plot_EU.png")

plt.show()


# For WorldWide
grouped_start_year_World = filter_natural_world.groupby('Start Year').size()


plt.figure(figsize=(20, 8))

plt.plot(grouped_start_year_World.index, grouped_start_year_World.values)

plt.xlabel('Start Year')
plt.ylabel('Number of Natural Disasters')
plt.title('Number of Natural Disasters WorldWide in each year')


plt.savefig("natural_disasters_all_year_plot_WW.png")

plt.show()


In [ ]:
# Plot occurences of all disaster type for EU and WW

# Defining the method

def plot_diaster_types(dataset, colors, string, title):
    p = figure(x_range=dataset.index.astype(str).tolist(), height=400, width=700, title=title,
            toolbar_location=None, tools="")

    disaster_types = dataset.columns.tolist()
    bar_width = 1 / len(disaster_types)  
    renderers = [] 

    for i, disaster_type in enumerate(disaster_types):
        x_pos = [year + i * bar_width for year in range(len(dataset))]
        source = ColumnDataSource(data=dict(x=x_pos, top=dataset[disaster_type], name=[disaster_type] * len(x_pos), year=dataset.index.astype(str).tolist()))

        renderer = p.vbar(x='x', top='top', width=bar_width, color=colors[disaster_type], source=source)
        renderers.append(renderer)  
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Number of Disasters'
    p.xaxis.major_label_orientation = np.pi / 4

    hover = HoverTool(tooltips=[("Year", "@year"), ("Disaster Type", "@name"), ("Occurrence", "@top")], renderers=renderers)
    p.add_tools(hover)

    legend_items = [(disaster_type, [renderer]) for disaster_type, renderer in zip(disaster_types, renderers)]
    legend = Legend(items=legend_items, location="top_right", click_policy='hide')
    p.add_layout(legend, 'right')

    output_file(string + ".html")
    show(p)


# Plot  for EU
grouped_year = filter_natural.groupby(['Start Year', 'Disaster Type']).size().unstack(fill_value=0)
plot_diaster_types(grouped_year, disaster_colors, "plot_disaster_EU_all_years", "Number of Natural Disasters by Type and Year in EU")

# Plot for WW 
grouped_year_world = filter_natural_world.groupby(['Start Year', 'Disaster Type']).size().unstack(fill_value=0)
plot_diaster_types(grouped_year_world, disaster_colors, "plot_disaster_worldwide_all_years", "Number of Natural Disasters by Type and Year WorldWide")


Giving the previous plots, we see the countries and their CO2 emission. In the next section, the countries producing the most and least amount of CO2 are being highligted. For EU we see that Germany and Italy are the countries with the highest CO2 emission, while Malta and Latvia are the countries with the lowest CO2 emission. For Worldwide er have Qatar and Kuwait as the two countries having the most CO2 emission, while Somalia and Congo being the counties with the lowest. 

There has been selected a joker that do not spike in the high or lower end but would have a CO2 emission level somewhere in the middle of all of the countries in EU and the Wordwide. For EU Sweden is the joker, and for the whole world China has been chosen. 

In [ ]:
# Plot for selected countries
grouped_country = filter_natural.groupby(['Country', 'Start Year', 'Disaster Type']).size().unstack(fill_value=0)

# Defining the method 
def disastertypes_country(dataset, countries, colors, name_of_file):
    plots = []

    num_countries = len(countries)
    plots_per_row = 2
    num_rows = (num_countries + plots_per_row - 1) // plots_per_row

    disaster_names = []

    for idx, country in enumerate(countries):
        country_data = dataset.loc[country]
        disaster_types = country_data.columns
        
        all_years = np.arange(country_data.index.min(), country_data.index.max() + 1)

        non_zero_disasters = [disaster_type for disaster_type in disaster_types if country_data[disaster_type].sum() > 0]

        p = figure(title=f'Number of Natural Disasters by Type and Year in {country}',
                    x_axis_label='Year', y_axis_label='Number of Disasters',
                    x_range=(min(all_years)-1, max(all_years)+1), width = 475, height = 250)

        plotted_disaster_types = []

        for i, disaster_type in enumerate(non_zero_disasters):
            x = all_years
            y = country_data[disaster_type].reindex(all_years, fill_value=0)
            source = ColumnDataSource(data=dict(x=x, y=y, name=[disaster_type] * len(x)))
            bars = p.vbar(x='x', top='y', width=0.8, color=colors[disaster_type], alpha=0.5, source=source)
            plotted_disaster_types.append((disaster_type, [bars]))
            disaster_names.append(disaster_type)  

        p.xaxis.ticker = all_years
        p.xaxis.major_label_orientation = "vertical"
        p.yaxis.ticker = np.arange(0, 6, 1)
        legend = Legend(items=plotted_disaster_types, location="top_left", click_policy="hide")
        legend.spacing = 5
        legend.label_text_font_size = "6pt"
        p.add_layout(legend, 'right')

        #tooltips = [("Year", "@x"), ("Disaster Type", "@name"), ("Occurrences", "@y")]
        # hover = HoverTool(tooltips=tooltips, mode="vline")
        # p.add_tools(hover)

        plots.append(p)

    grid = gridplot(plots, ncols=plots_per_row)

    output_file(name_of_file + ".html")
    show(grid)

    
#PLOTS FOR EU

#1. Highest CO2 emission: Germany
countries_to_include_estonia = ['Germany']
disastertypes_country(grouped_country, countries_to_include_estonia, disaster_colors, "Germany_EU")


#2. Highest CO2 emission: Italy 
countries_to_include_sweden = ['Italy']
disastertypes_country(grouped_country, countries_to_include_sweden, disaster_colors, "Italy_EU")


# 1. Lowest CO2 emission: Malta
countries_to_include_malta = ['Malta']
disastertypes_country(grouped_country, countries_to_include_malta, disaster_colors, "Malta_EU")

# 2. Lowest CO2 emission: Latvia
countries_to_include_malta = ['Latvia']
disastertypes_country(grouped_country, countries_to_include_malta, disaster_colors, "Latvia_EU")

#Joker
countries_to_include_italy = ['Sweden']
disastertypes_country(grouped_country, countries_to_include_italy, disaster_colors, "Sweden_EU")


# PLOTS FOR WW 
grouped_country_WW = filter_natural_world.groupby(['Country', 'Start Year', 'Disaster Type']).size().unstack(fill_value=0)

# Qatar, Kuwait cannot be found with the highest CO2 is not in the dataset 

# 1.Lowest CO2 emission: Somalia
countries_to_include_WW_Somalia = ['Somalia']
disastertypes_country(grouped_country_WW, countries_to_include_WW_Somalia, disaster_colors, "Somalia_WW")

#2.Lowest CO2 emission: Congo
countries_to_include_WW_Congo = ['Congo']
disastertypes_country(grouped_country_WW, countries_to_include_WW_Congo, disaster_colors, "Congo_WW")


# Joker
countries_to_include_WW_China = ['China']
disastertypes_country(grouped_country_WW, countries_to_include_WW_China, disaster_colors, "China_WW")


